# SOHO analysis 
The purpose of this notebook is to analysis visual-spectrum images of the sun from the SOHO imaging satellite at the L1 Lagrange point (meaning the images are from the same perspective as the Earth) and use this data to determine the rotational period of the sun.

To do:

- [ ] README.txt
   
    ~~--SDO/HMI satellite~~

    ~~--using JSOC API for data access~~
    ~~-- using VSO api through sunpy~~
~~- [ ] Make file importing work~~
- [ ] Perform analysis on data
    - [ ] Plot velocity for each point with error bars
    - [ ] Plot velocity against latitude
    - [ ] Plot velocity against longitude (possible warping)
    - [ ] Plot velocity against longitude and latitude (polar coords) and hopefully reconstruct an image of the sun with velocities.
    - [ ] Interpolations?

In [1]:
#imports
import os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
from ipywidgets import interact, Dropdown
import cv2
from astropy.coordinates import SkyCoord
import json

from utils.data import fetch_images, get_files_with_times
from utils.image_processing import detect_sunspots
from utils.feature_tracking import SunspotTracker

In [2]:
#Parameters
data_bank_url = "https://soho.nascom.nasa.gov/data/REPROCESSING/Completed/2025/hmiigr/"
save_dir = "sdo_hmi_jpgs"
start_date = datetime(2025,4,22,0,0)
end_date = datetime(2025,5,7,0,0)
cadence = timedelta(hours=1.5)

#Download the images
fetch_images(
    data_bank_url, 
    save_dir, 
    start_date, 
    end_date, 
    cadence,
    )

#Collect the images
file_paths, times = get_files_with_times(save_dir)


Achtung! Image not available: https://soho.nascom.nasa.gov/data/REPROCESSING/Completed/2025/hmiigr/20250427/20250427_2230_hmiigr_512.jpg


Error at 2025-04-28 00:00:00: HTTPSConnectionPool(host='soho.nascom.nasa.gov', port=443): Read timed out. (read timeout=10)


successfully downloaded sdo_hmi_jpgs/20250506/20250506_0000.jpg
successfully downloaded sdo_hmi_jpgs/20250506/20250506_0130.jpg
successfully downloaded sdo_hmi_jpgs/20250506/20250506_0300.jpg
successfully downloaded sdo_hmi_jpgs/20250506/20250506_0430.jpg


successfully downloaded sdo_hmi_jpgs/20250506/20250506_0600.jpg


successfully downloaded sdo_hmi_jpgs/20250506/20250506_0730.jpg
successfully downloaded sdo_hmi_jpgs/20250506/20250506_0900.jpg
successfully downloaded sdo_hmi_jpgs/20250506/20250506_1030.jpg
successfully downloaded sdo_hmi_jpgs/20250506/20250506_1200.jpg


successfully downloaded sdo_hmi_jpgs/20250506/20250506_1330.jpg


successfully downloaded sdo_hmi_jpgs/20250506/20250506_1500.jpg
successfully downloaded sdo_hmi_jpgs/20250506/20250506_1630.jpg
successfully downloaded sdo_hmi_jpgs/20250506/20250506_1800.jpg
successfully downloaded sdo_hmi_jpgs/20250506/20250506_1930.jpg
successfully downloaded sdo_hmi_jpgs/20250506/20250506_2100.jpg


successfully downloaded sdo_hmi_jpgs/20250506/20250506_2230.jpg
All requested files successfully downloaded!


The first step of data processing is preprocessing and identification of the sunspots. This is done via the utility `image_processing.py`. The utility uses contouring to identify the centroids of features on the sun which *should* correspond well to the sunspots.

In [9]:
# Create a dropdown to select days
day_dirs = sorted([d for d in os.listdir("sdo_hmi_jpgs") if os.path.isdir(os.path.join("sdo_hmi_jpgs", d))])
@interact(day=Dropdown(options=day_dirs, description="Select Day:"))
def show_day_images(day):
    day_path = os.path.join("sdo_hmi_jpgs", day)
    files = sorted([f for f in os.listdir(day_path) if f.endswith(".jpg")])[:16] #Only the first 12 images
    
    fig, axes = plt.subplots(3, 4, figsize=(15, 10))
    for ax, file in zip(axes.flat, files):
        img, centroids, _, _ = detect_sunspots(os.path.join(day_path, file))
        print(centroids)
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax.scatter([c[0] for c in centroids], [c[1] for c in centroids], s=5, c='blue')
        ax.set_title(file.split('_')[1])  # Show time (hhmm)
        ax.axis('off')
    plt.tight_layout()

interactive(children=(Dropdown(description='Select Day:', options=('20250422', '20250423', '20250424', '202504…

The visualization above includes a drop down to select the day of choice and then the image represents all data (including coordinates at the top) collected from that date. The sunspot centroids are shown via blue dots.

---

The next part of the data analysis involves tracking the sunspots between frames. This was quite a complicated process with a decent amount of trial and error, but the main issue I faced was actually an error in using Carrington coordinates instead of Stony coordinates. This meant that all of my velocities were centered around 0 as Carrington is a rotating reference frame where the same point on the sun would move as the Earth orbited.

In [4]:
'''
This block computes the longitudinal angular-velocity of the sunspots between different
images via a nearst neighbour algorithm. The data is saved in a JSON file called sunspot_data.json.
'''

def skycoord_to_dict(coord):
    #This function is for saving to JSON
    return {
        'lon': coord.lon.deg,
        'lat': coord.lat.deg,
        'frame': coord.frame.name,
        'unit': 'deg'
    }

_, _, solar_center, solar_radius = detect_sunspots(file_paths[0]) #Initial value for solar radius from the first image
tracker = SunspotTracker(solar_center, solar_radius, 1)

#main feature tracking loop
for img, time in zip(file_paths, times):
    img, centroids, solar_center, solar_radius = detect_sunspots(img)
    
    tracker.process_frame(time, centroids)

#Filter the data to only include tracks with one or more velocities
filtered_tracks = [t for t in tracker.tracks if len(t['velocities']) >= 1]

###Write the data to JSON
#First convert necessary datatypes to str
for entry in filtered_tracks:
    if 'times' in entry:
        entry['times'] = [t.isoformat() if isinstance(t, datetime) else t for t in entry['times']]
    if 'positions_helio' in entry:
        entry['positions_helio'] = [skycoord_to_dict(coord) if isinstance(coord, SkyCoord) else coord for coord in entry['positions_helio']]

with open(file='sunspot_data.json',mode='w') as f:
    json.dump(filtered_tracks, f, indent=4)

The data is saved into a JSON file and then the following codeblock is used to extract the data again.

In [5]:
'''
This code is to recover the data from the JSON
'''

def dict_to_skycoord(d):
    return SkyCoord(lon=d['lon'], lat=d['lat'],
                    frame=d['frame'], unit=d['unit'])

with open("sunspot_data.json", 'r') as f:
    data = json.load(f)

#Convert strings back into necessary datatypes
for entry in data:
    if 'times' in entry:
        entry['times'] = [datetime.fromisoformat(t) if isinstance(t, str) else t for t in entry['times']]
    if 'positions_helio' in entry:
        entry['positions_helio'] = [dict_to_skycoord(coord) if isinstance(coord, SkyCoord) else coord for coord in entry['positions_helio']]


We can now use the same visualization code from before and match the coordinates of specific identified features with the centroids. This method is not perfect as I used a rudamentary, greedy assignment method. Possible improvements would be a 1:1 matching method rather than a FIFO approach; using velocity as a predictor and using precision markers (closest wins the identifier); Merge short tracks (although it'd be hard to know which to merge); and adding a Kalman filter for predictive smoothing. 

Aside from that, the identifiers can be visualized below.

In [8]:
# Create a dropdown to select days
day_dirs = sorted([d for d in os.listdir("sdo_hmi_jpgs") if os.path.isdir(os.path.join("sdo_hmi_jpgs", d))])
@interact(day=Dropdown(options=day_dirs, description="Select Day:"))
def show_day_images(day):
    day_path = os.path.join("sdo_hmi_jpgs", day)
    files = sorted([f for f in os.listdir(day_path) if f.endswith(".jpg")])[:16] #Only the first 12 images
    
    fig, axes = plt.subplots(3, 4, figsize=(15, 10))
    for ax, file in zip(axes.flat, files):
        img, centroids, _, _ = detect_sunspots(os.path.join(day_path, file))
        print(centroids)
        
        ax.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
        ax.scatter([c[0] for c in centroids], [c[1] for c in centroids], s=5, c='red')
        matches = []
        for i, coords in enumerate(centroids):
            for track_idx, track in enumerate(data):
                for j, pos_px in enumerate(track['positions_px']):
                    if list(coords) == pos_px:
                        matches.append([i,track_idx])
        
        for centroid_idx, track_id in matches:
            x,y = centroids[centroid_idx]
            ax.text(x + 3, y + 3, str(track_id), color='blue', fontsize=8, weight='bold')
            
        
        
        ax.set_title(file.split('_')[1])  # Show time (hhmm)
        ax.axis('off')
    plt.tight_layout()

interactive(children=(Dropdown(description='Select Day:', options=('20250422', '20250423', '20250424', '202504…

Notice that the same spot often jumps to another identifier. This shouldn't be too much of an issue as you can see below, the average track length is 4—which is a pretty decent sample size.

In [7]:
#Number of datapoints for each feature
lengths = []
for i,entry in enumerate(data):
    print(f"point {i}: len = {len(entry['velocities'])}")
    lengths.append(len(entry))
print(sum(lengths)/len(lengths))

point 0: len = 10
point 1: len = 1
point 2: len = 14
point 3: len = 13
point 4: len = 2
point 5: len = 2
point 6: len = 3
point 7: len = 11
point 8: len = 11
point 9: len = 3
point 10: len = 6
point 11: len = 8
point 12: len = 1
point 13: len = 2
point 14: len = 2
point 15: len = 3
point 16: len = 6
point 17: len = 2
point 18: len = 15
point 19: len = 2
point 20: len = 8
point 21: len = 1
point 22: len = 3
point 23: len = 1
point 24: len = 1
point 25: len = 1
point 26: len = 3
point 27: len = 3
point 28: len = 1
point 29: len = 1
point 30: len = 1
point 31: len = 4
point 32: len = 1
point 33: len = 2
point 34: len = 2
point 35: len = 1
point 36: len = 3
point 37: len = 2
point 38: len = 2
point 39: len = 3
point 40: len = 1
point 41: len = 1
point 42: len = 7
point 43: len = 1
point 44: len = 1
point 45: len = 2
point 46: len = 1
point 47: len = 2
point 48: len = 2
point 49: len = 1
point 50: len = 10
point 51: len = 3
point 52: len = 6
point 53: len = 2
point 54: len = 5
point 55: len

So with 117 detected features with atleast one velocity datapoint, we can start the data analysis. Everything up to this point was extracting velocity data. There are many improvements which could be done (as stated in a markdown above the visualization with the identifiers for the features.

Things to try:
- Plotting angular velocity by latitude